In [ ]:
# ---------------------------------- Libraries ----------------------------------
!pip install -q transformers accelerate torch datasets peft trl openai scikit-learn gradio
!pip install -U huggingface_hub
!pip install -U bitsandbytes
!pip install --upgrade trl

In [ ]:

# ---------------------------------- Imports ----------------------------------
import bitsandbytes as bnb
from datasets import load_dataset, Dataset, DatasetDict
import gc
from google.colab import userdata
from huggingface_hub import create_repo, login, HfApi, notebook_login
from IPython.display import display, Markdown
import json
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import time
import torch
import trl
from tqdm.notebook import tqdm
import transformers
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    BitsAndBytesConfig,
)
from trl import SFTTrainer


In [ ]:
# ---------------------------------- HuggingFace Connection ----------------------------------
hf_token = os.environ.get('HF_TOKEN') or userdata.get('HF_TOKEN')

if hf_token:
    login(token=hf_token)
    print("HuggingFace login successful.")
else:
    print("HuggingFace token not found. Please set the HF_TOKEN environment variable or store it in Colab secrets.")

In [ ]:
# ---------------------------------- Use GPU ----------------------------------
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    # Set default device to GPU
    torch.set_default_device("cuda")
    print("PyTorch default device set to CUDA (GPU).")
else:
    print("WARNING: No GPU detected. Running on CPU")

In [ ]:
# ---------------------------------- Functions ----------------------------------
# Helper function for markdown display
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))



In [ ]:
# ---------------------------------- Load Dataset ----------------------------------
# https://huggingface.co/datasets/Daniel-ML/sentiment-analysis-for-financial-news-v2/viewer
dataset_id = "Daniel-ML/sentiment-analysis-for-financial-news-v2"

# Load the data set
labeled_dataset = load_dataset(dataset_id, split = "train")
print("Dataset loaded successfully!")


In [ ]:
# ---------------------------------- View Dataset ----------------------------------
# View the data set
print("\n--- Dataset Information ---")
print(labeled_dataset)

# View the dataset features
print("\n--- Dataset Features ---")
print(labeled_dataset.features)

# Unique Labels
labels = labeled_dataset.to_pandas()['sentiment'].unique().tolist()
print("\n--- Unique Labels ---")
print(f"Unique labels in the dataset: {labels}")




--- Dataset Information ---  
Dataset({  
    features: ['sentiment', 'text'],  
    num_rows: 4846  
})  
  
--- Dataset Features ---  
{'sentiment': Value('string'), 'text': Value('string')}  
  
--- Unique Labels ---  
Unique labels in the dataset: ['neutral', 'negative', 'positive']  


In [ ]:
# View the data as a python DataFrame
display(labeled_dataset.select(range(5)).to_pandas()[["text", "sentiment"]])

**text**  **sentiment**  
**0**  According to Gran , the company has no plans t...  neutral  
**1**  Technopolis plans to develop in stages an area...  neutral  
**2**  The international electronic industry company ...  negative  
**3**  With the new production plant the company woul...  positive  
**4**  According to the company 's updated strategy f...  positive  

In [ ]:
# ---------------------------------- Split Dataset into Train / Test Dataset ----------------------------------
# Split the Data into Train and Test Sets ---
print("\nSplitting data into Train (90%) and Test (10%)...")
train_test_split_ratio = 0.10

# Set seed for reproducability
seed = 42

# Using datasets built-in method.  Shuffle the data to allow the LLM not learn the order of each sample.
split_dataset = labeled_dataset.train_test_split(
    test_size = train_test_split_ratio, seed = seed, shuffle = True,
)
train_dataset = split_dataset["train"]
test_dataset = split_dataset["test"]

print(f"Training set size: {len(train_dataset)}")
print(f"Test set size: {len(test_dataset)}")
print("\nTrain/Test Split Complete.")




Splitting data into Train (90%) and Test (10%)...  
Training set size: 4361  
Test set size: 485  
  
Train/Test Split Complete.  


In [ ]:
# ---------------------------------- View Dataset Balance ----------------------------------
sns.countplot(labeled_dataset["sentiment"])

If the dataset is balanced, the model may have a bias towards certain data.

In [ ]:
# ---------------------------------- Format for Supervised Fine-Tuning ("google/gemma-3-1b-it") ----------------------------------
def format_for_sft_gemma(example, tokenizer):

    # Define the conversation structure
    system_prompt = "Classify the sentiment of the following sentence from News as positive, negative, or neutral."
    user_prompt = f"Sentence: {example['text']}"
    assistant_response = example['sentiment'] # The target label

    messages = [
        {"role": "user", "content": f"{system_prompt}\n{user_prompt}"}, # Combine system/user for simplicity here
        {"role": "assistant", "content": assistant_response}
    ]
    # Apply the tokenizer's chat template.
    # tokenize = False: means we want the text output, not token IDs.
    # add_generation_prompt = False: means we’re NOT adding the assistant prompt to generate the response. We already provided the assistant's message
    formatted_text = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
    return {"text": formatted_text}

In [ ]:
# ---------------------------------- Test the Supervised Fine-Tuning Function ("google/gemma-3-1b-it") ----------------------------------
os_model_id = "google/gemma-3-1b-it"

# Define the quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # Use 4-bit weights (saves lots of RAM)
    bnb_4bit_quant_type = "nf4", # nf4 is a better 4-bit format (Non-Float 4)
    bnb_4bit_compute_dtype = torch.float16 # Math is done in float16 for speed
)

# Load the tokenizer to convert text to tokens (numbers)
base_os_tokenizer = AutoTokenizer.from_pretrained(os_model_id)

# Set pad token if missing (Gemma often doesn't have one)
# A pad token is a special token used to make all input sequences the same length in a batch.
if base_os_tokenizer.pad_token is None:
    base_os_tokenizer.pad_token = base_os_tokenizer.eos_token
    print(f"Set pad_token to eos_token ({base_os_tokenizer.eos_token})")


In [ ]:
# ---------------------------------- Test Random Data ("google/gemma-3-1b-it") ----------------------------------
# Sample input dictionary
# <bos>                      → Beginning of sequence (tells the model: "Start reading")
# <start_of_turn>user        → Start of the user's turn
# Classify the sentiment...  → Instruction and user input
# <end_of_turn>              → End of user input
# <start_of_turn>model       → Start of the assistant/model's reply
# positive                   → The expected response
# <end_of_turn>              → End of assistant reply

example = {
    "text": "The economy is showing signs of recovery after a tough year.",
    "sentiment": "positive"
}

# Apply the formatting function
formatted_example = format_for_sft_gemma(example, base_os_tokenizer)

# Print the result
print("\n--- Sample Formatted Prompt ---")
print(formatted_example["text"])

In [ ]:
# ---------------------------------- Format for Supervised Fine-Tuning ("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B") ----------------------------------
def format_for_sft_deepseek_qwen(example, tokenizer):
    # Target task
    system_prompt = "Classify the sentiment of the following sentence from News as positive, negative, or neutral."
    user_prompt = f"Sentence: {example['text']}"
    assistant_response = example['sentiment']  # gold label

    # Qwen-style roles (ChatML)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
        {"role": "assistant", "content": assistant_response},
    ]

    # For SFT we want the full conversation (incl. assistant target) serialized as text
    # so we set tokenize=False and add_generation_prompt=False.
    formatted_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )
    return {"text": formatted_text}

In [ ]:
# ---------------------------------- Test the Supervised Fine-Tuning Function ("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B") ----------------------------------
os_model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Define the quantization configuration
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # Use 4-bit weights (saves lots of RAM)
    bnb_4bit_quant_type = "nf4", # nf4 is a better 4-bit format (Non-Float 4)
    bnb_4bit_compute_dtype = torch.float16 # Math is done in float16 for speed
)

# Load the tokenizer to convert text to tokens (numbers)
base_os_tokenizer = AutoTokenizer.from_pretrained(os_model_id)

# Set pad token if missing (Gemma often doesn't have one)
# A pad token is a special token used to make all input sequences the same length in a batch.
if base_os_tokenizer.pad_token is None:
    base_os_tokenizer.pad_token = base_os_tokenizer.eos_token
    print(f"Set pad_token to eos_token ({base_os_tokenizer.eos_token})")


In [ ]:
# ---------------------------------- Test A Random Row ("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B") ----------------------------------
# Pick a random index
idx = random.randint(0, len(train_dataset) - 1)

# Retrieve that row
random_row = train_dataset[idx]

print(random_row)

example_2 = {
    "text": random_row['text'],
    "sentiment": random_row['sentiment']
}

# Apply the formatting function
formatted_example_2 = format_for_sft_deepseek_qwen(example_2, base_os_tokenizer)

# Print the result
print("\n--- Sample Formatted Prompt ---")
print(formatted_example_2["text"])

In [ ]:
# ---------------------------------- Zero Shot Classification ("google/gemma-3-1b-it") ----------------------------------
os_model_id = "google/gemma-3-1b-it"

base_os_tokenizer = AutoTokenizer.from_pretrained(os_model_id, use_fast=True)
if base_os_tokenizer.pad_token is None:
    base_os_tokenizer.pad_token = base_os_tokenizer.eos_token

base_os_model = AutoModelForCausalLM.from_pretrained(
    os_model_id,
    device_map="auto",
    torch_dtype="auto",
    low_cpu_mem_usage=True,
)

base_os_model.config.pad_token_id = base_os_tokenizer.pad_token_id
print("Loaded Gemma-3 1B-IT without bnb 4-bit.")


In [ ]:
# ---------------------------------- Apply Formatting for SFT Function to Test/Train Data ("google/gemma-3-1b-it") ----------------------------------
print("\nFormatting data for SFTTrainer (Gemma format)...")
sft_train_dataset = train_dataset.map(
              format_for_sft_gemma,
              fn_kwargs={"tokenizer": base_os_tokenizer},
              remove_columns=list(train_dataset.features)
              )
sft_test_dataset = test_dataset.map(
              format_for_sft_gemma,
              fn_kwargs={"tokenizer": base_os_tokenizer},
              remove_columns=list(test_dataset.features)
              )
print("SFTTrainer (Gemma) formatting complete.")
print("Sample SFT Gemma format:")
print(sft_train_dataset[0]['text'])

In [ ]:
# ---------------------------------- Zero-Shot Classification Prompt Function ("google/gemma-3-1b-it") ----------------------------------
def create_zeroshot_prompt_gemma(sentence, tokenizer):

    """Creates zero-shot prompt using Gemma chat template."""
    system_prompt = f"Classify the sentiment of the following sentence from Financial News. Respond with ONLY ONE of the following labels: {', '.join(labels)}."
    user_prompt = f"Sentence: {sentence}"
    messages = [
        {"role": "user", "content": f"{system_prompt}\n{user_prompt}"},
    ]
    # Apply template, add_generation_prompt=True adds the assistant turn marker
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [ ]:
# ---------------------------------- Inference Function for Zero-Shot Classification ("google/gemma-3-1b-it") ----------------------------------
# This function runs inference using the prompt generated above and returns the predicted sentiment label.
# It Gets the formatted prompt using the first function "create_zeroshot_prompt_gemma".
# It then tokenizes the prompt and sends it to the appropriate device (e.g., GPU):
def classify_zero_shot_os_gemma(sentence, model, tokenizer):

    prompt = create_zeroshot_prompt_gemma(sentence, tokenizer)
    inputs = tokenizer(prompt, return_tensors = "pt", truncation = True, max_length = 512).to(model.device)

    eos_id = tokenizer.eos_token_id          # eos_token_id: end of sentence
    pad_id = tokenizer.pad_token_id          # pad_token_id: padding token

    # Run the model to generate output, we are not performing gradient tracking since it's only inference!
    with torch.no_grad():

      outputs = model.generate(
          **inputs,
          max_new_tokens = 10, # Limits the number of tokens the model can generate
          eos_token_id = eos_id,
          pad_token_id = eos_id,
          do_sample = False
      )
    # Extracts and decode the generated text
    response_ids = outputs[0][inputs['input_ids'].shape[1]:]
    response_text = tokenizer.decode(response_ids, skip_special_tokens=True).strip()

    # print(f"Generated Text (Gemma): {response_text}")

    # Clean and Validate Output
    labels = ["neutral", "negative", "positive"]

    predicted_label = "Unknown"
    for label_text in labels:
        if label_text.lower() in response_text.lower(): # Simple check if label is present
              predicted_label = label_text
              break

    print(f"Predicted Label: {predicted_label}")
    return predicted_label


In [ ]:
# ---------------------------------- Extract True Labels ("google/gemma-3-1b-it") ----------------------------------
true_labels = []

# Extract the ground truth (target output) which represents the True class
true_labels = [ex['sentiment'] for ex in test_dataset]
true_labels

In [ ]:
# ---------------------------------- Evaluate on Test Set ("google/gemma-3-1b-it") ----------------------------------
base_os_predictions = [] # Renaming for clarity if needed, but reusing is fine

# Invoke the pre-trained Gemma LLM on all testing datasets
for example in tqdm(test_dataset):
    predicted_label = classify_zero_shot_os_gemma(example['text'], base_os_model, base_os_tokenizer)
    base_os_predictions.append(predicted_label)

In [ ]:
# ---------------------------------- Examine Zero-Shot Evaluation Results ("google/gemma-3-1b-it") ----------------------------------
valid_indices = [i for i, p in enumerate(base_os_predictions) if p not in ["Error", "Unknown"]]
filtered_preds = [base_os_predictions[i] for i in valid_indices]
filtered_true = [true_labels[i] for i in valid_indices]

accuracy = accuracy_score(filtered_true, filtered_preds)
report = classification_report(filtered_true, filtered_preds, labels=labels, zero_division=0, target_names=labels)
print(f"Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(report)


In [ ]:
# ---------------------------------- Visualize Confusion Matrix ("google/gemma-3-1b-it") ----------------------------------
# Compute confusion matrix
cm = confusion_matrix(filtered_true, filtered_preds, labels=labels)

# Create heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels, cbar=False)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix - Base Gemma Model (Zero-Shot)")
plt.tight_layout()
plt.show()

# This confusion matrix shows that the model wasn't the best at predicting the true classification

In [ ]:
# ---------------------------------- LoRA ----------------------------------
# --- Prepare model for k-bit training (important for quantized models) ---
base_os_model.gradient_checkpointing_enable()  # Saves memory during training
prepared_model = prepare_model_for_kbit_training(base_os_model)
print("Model prepared for k-bit training.")

print(prepared_model)


# --- LoRA Configuration ---
# Target modules often include query/key/value layers in self-attention blocks
# This depends on the model architecture (use print(prepared_model) to inspect layers)
# For Qwen-based models, common targets might be 'q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'
# Let's start with a reasonable default set
lora_config = LoraConfig(
    r = 16,  # LoRA rank (dimension of adapter matrices). Higher rank = more parameters, potentially better fit but slower. 8, 16, 32 are common.
    lora_alpha = 32,  # Scaling factor for LoRA weights (often 2*r).
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],  # Modules to apply LoRA to.
    lora_dropout = 0.05,  # Dropout probability for LoRA layers.
    bias = "none",  # Usually set to 'none'.
    task_type = "CAUSAL_LM",  # Task type for sequence generation.
)
print("LoRA Config created.")

# --- Apply PEFT to the model ---
# This adds the small LoRA layers into the big model. During fine-tuning, only these are updated — saving a lot of compute and memory
peft_model = get_peft_model(prepared_model, lora_config)



# The percentage (often < 1%) is small because LoRA only introduces and trains the parameters within the small adapter layers (`q_proj`, `k_proj`, etc. in our config),
# It does not train the original billions of parameters in the base model (which remain frozen).
# Training far fewer parameters requires significantly less GPU memory (VRAM), making it feasible to fine-tune large models on hardware like Colab's T4 GPU.
# It's also much faster than full fine-tuning.
print("PEFT model created.")
peft_model.print_trainable_parameters()  # See how few parameters we're actually training!


In [ ]:
# ---------------------------------- Training Arguments ----------------------------------
output_dir = "./sentiment_finetuned_adapter"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=3,
    logging_steps=25,
    save_strategy="epoch",
    # optim="paged_adamw_8bit",
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="none",
    dataloader_pin_memory=False,   # avoid pinning CUDA tensors
    dataloader_num_workers=0,      # simplest/safer for Colab-style envs
)

print("Training Arguments set.")

# ---------------- Safety: ensure tokenizer & model padding are set ----------------
if getattr(base_os_tokenizer, "pad_token", None) is None:
    base_os_tokenizer.pad_token = base_os_tokenizer.eos_token

if hasattr(peft_model, "config"):
    peft_model.config.pad_token_id = base_os_tokenizer.pad_token_id
    peft_model.config.eos_token_id = base_os_tokenizer.eos_token_id
    if getattr(peft_model.config, "use_cache", None) is not None:
        peft_model.config.use_cache = False

# ---------------- Build a text-only data collator ----------------
data_collator = DataCollatorForLanguageModeling(
    tokenizer=base_os_tokenizer,
    mlm=False
)

# ---------------- Prepare / normalize the dataset ----------------
TEXT_COL = "text"  # change if your column is different

# 1) If it's a DatasetDict, pick the train split
if isinstance(sft_train_dataset, DatasetDict):
    if "train" not in sft_train_dataset:
        raise ValueError(f"DatasetDict has splits {list(sft_train_dataset.keys())}, but no 'train' split.")
    sft_train_dataset = sft_train_dataset["train"]

# 2) If it's a list of dicts, wrap it
if not hasattr(sft_train_dataset, "column_names"):
    sft_train_dataset = Dataset.from_list(sft_train_dataset)

assert TEXT_COL in sft_train_dataset.column_names, f"Expected '{TEXT_COL}' column in dataset; found {sft_train_dataset.column_names}"
assert len(sft_train_dataset) > 0, "Train dataset is empty."

# ---------------- Tokenize ----------------
MAX_LEN = 2048  # adjust to your context window & memory

def _tok_fn(batch):
    # Return CPU-side lists; collator will create tensors and pad dynamically
    return base_os_tokenizer(
        batch[TEXT_COL],
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
        return_attention_mask=True
    )

tokenized_train = sft_train_dataset.map(
    _tok_fn,
    batched=True,
    remove_columns=sft_train_dataset.column_names  # drop raw text + other cols
)

# Reset any prior "torch" formatting or CUDA device formatting on the dataset
# so that we yield CPU-native python lists to the collator.
if hasattr(tokenized_train, "set_format"):
    tokenized_train.set_format(type=None)

# Sanity checks
required_cols = {"input_ids", "attention_mask"}
missing = required_cols.difference(set(tokenized_train.column_names))
if missing:
    raise ValueError(f"Tokenized dataset missing columns: {missing}. Columns present: {tokenized_train.column_names}")
if len(tokenized_train) == 0:
    raise ValueError("Tokenized train dataset is empty after mapping.")

print(f"Tokenized train samples: {len(tokenized_train)}")

# ---------------- Initialize vanilla HF Trainer (no TRL) ----------------
from transformers import Trainer # Import Trainer class

trainer = Trainer(
    model=peft_model,                    # LoRA-wrapped model
    args=training_args,
    train_dataset=tokenized_train,       # <-- ensure this is a Dataset, not DatasetDict
    tokenizer=base_os_tokenizer,         # OK despite deprecation; HF 5.x will prefer processing_class
    data_collator=data_collator,
)

print("Trainer initialized (no SFTTrainer, no AutoProcessor).")

# ---------------- Train ----------------
print("\n--- Starting Fine-tuning... ---")
try:
    training_results = trainer.train()
    print("--- Fine-tuning Complete! ---")
    print(training_results)

    # --- Save the LoRA Adapter + tokenizer ---
    print(f"Saving LoRA adapter model to {output_dir}...")
    trainer.save_model(output_dir)           # Saves adapter weights for PEFT-wrapped model
    base_os_tokenizer.save_pretrained(output_dir)
    print("Adapter and tokenizer saved.")

    # --- Clean up memory ---
    del trainer
    del peft_model
    torch.cuda.empty_cache()
    gc.collect()
    print("Cleaned up training objects from memory.")

    fine_tuning_successful = True

except Exception as e:
    print(f"Error during fine-tuning: {e}")
    fine_tuning_successful = False

In [ ]:
# ---------------------------------- Push to HuggingFace ----------------------------------
from huggingface_hub import create_repo, HfApi, HfFolder

!huggingface-cli login

model_name = "gemma-sentiment-lora"
username = "david125tran"

# Create or get your repo (won’t error if it already exists)
repo_id = f"{username}/{model_name}"
create_repo(repo_id, private=False, exist_ok=True)

# Push the LoRA adapter + tokenizer folder
api = HfApi()
api.upload_folder(
    folder_path="./sentiment_finetuned_adapter",  # where your adapter and tokenizer are saved
    repo_id=repo_id,
    commit_message="Upload Gemma 3B sentiment LoRA adapter 🎉",
)

print(f"✅ Successfully uploaded! View it here: https://huggingface.co/{repo_id}")


In [ ]:
# ---------------------------------- Evaluate Model ----------------------------------
# ==== Config you may tweak ====
os_model_id = os_model_id  # e.g. "google/gemma-3-1b-it"
adapter_path = "./sentiment_finetuned_adapter"
TEXT_COL = "text"
labels = labels

# ==== Imports ====
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from peft import PeftModel
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

# --------- Helper: pick best available quantization/backend ---------
def build_base_model(model_id: str):
    has_cuda = torch.cuda.is_available()
    # try 4-bit
    if has_cuda:
        try:
            import bitsandbytes as bnb  # noqa: F401
            qconf = BitsAndBytesConfig(load_in_4bit=True)
            print("Loading base model in 4-bit quantization…")
            return AutoModelForCausalLM.from_pretrained(
                model_id,
                quantization_config=qconf,
                device_map="auto",
                trust_remote_code=True,
            )
        except Exception as e:
            print(f"⚠️ 4-bit load failed: {e}")
            # try 8-bit
            try:
                print("Falling back to 8-bit quantization…")
                qconf = BitsAndBytesConfig(load_in_8bit=True)
                return AutoModelForCausalLM.from_pretrained(
                    model_id,
                    quantization_config=qconf,
                    device_map="auto",
                    trust_remote_code=True,
                )
            except Exception as e2:
                print(f"⚠️ 8-bit load failed: {e2}")

    # final fallback: full precision (bf16 if supported, else fp16/cpu)
    dtype = torch.bfloat16 if torch.cuda.is_bf16_supported() else (torch.float16 if has_cuda else torch.float32)
    device_map = "auto" if has_cuda else None
    print(f"Falling back to full precision dtype={dtype} (device_map={device_map})…")
    return AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=dtype,
        device_map=device_map,
        trust_remote_code=True,
    )

# --------- Load tokenizer (from your saved adapter dir) ---------
print(f"\n--- Loading Fine-tuned Open Source Model from {adapter_path} ---")
tuned_os_tokenizer = AutoTokenizer.from_pretrained(adapter_path, trust_remote_code=True)
if getattr(tuned_os_tokenizer, "pad_token", None) is None and getattr(tuned_os_tokenizer, "eos_token", None) is not None:
    tuned_os_tokenizer.pad_token = tuned_os_tokenizer.eos_token

# --------- Load base model with best available quantization ---------
base_model_reload = build_base_model(os_model_id)
if tuned_os_tokenizer.pad_token_id is not None and hasattr(base_model_reload, "config"):
    base_model_reload.config.pad_token_id = tuned_os_tokenizer.pad_token_id
    if getattr(base_model_reload.config, "use_cache", None) is not None:
        base_model_reload.config.use_cache = False  # avoids gradient checkpointing warnings if enabled elsewhere

# --------- Load PEFT adapter (no merge) ---------
tuned_os_model = PeftModel.from_pretrained(base_model_reload, adapter_path)
tuned_os_model.eval()
print("✅ Loaded PEFT model (base + adapter).")

# --------- Ensure ground-truth labels exist ---------
if "true_labels" not in locals() or not true_labels:
    print("Warning: true_labels not found from previous steps. Re-extracting from test_dataset…")
    true_labels = [ex["sentiment"] for ex in test_dataset]

# --------- Evaluation loop (no_grad + AMP where applicable) ---------
tuned_os_predictions = []
use_bf16 = torch.cuda.is_available() and torch.cuda.is_bf16_supported()
use_fp16 = torch.cuda.is_available() and not use_bf16

autocast_dtype = torch.bfloat16 if use_bf16 else (torch.float16 if use_fp16 else None)
autocast_ctx = (torch.autocast(device_type="cuda", dtype=autocast_dtype) if autocast_dtype is not None
                else torch.no_grad())  # will be replaced below for clarity

print("\n--- Evaluating fine-tuned model on test set ---")
with torch.no_grad():
    if autocast_dtype is not None:
        # AMP path
        with torch.autocast(device_type="cuda", dtype=autocast_dtype):
            for example in tqdm(test_dataset):
                sentence = example["text"]
                pred = classify_zero_shot_os_gemma(sentence, tuned_os_model, tuned_os_tokenizer)
                tuned_os_predictions.append(pred)
    else:
        # CPU / no AMP
        for example in tqdm(test_dataset):
            sentence = example["text"]
            pred = classify_zero_shot_os_gemma(sentence, tuned_os_model, tuned_os_tokenizer)
            tuned_os_predictions.append(pred)

# --------- Metrics ---------
print("\n--- Fine-Tuned Open Source Model: Evaluation Results ---")
valid_indices_tuned = [i for i, p in enumerate(tuned_os_predictions) if p not in ["Error", "Unknown"]]
filtered_preds_tuned = [tuned_os_predictions[i] for i in valid_indices_tuned]
filtered_true_tuned = [true_labels[i] for i in valid_indices_tuned]

accuracy_tuned = accuracy_score(filtered_true_tuned, filtered_preds_tuned)
report_tuned = classification_report(filtered_true_tuned, filtered_preds_tuned, labels=labels, zero_division=0)

print(f"Accuracy: {accuracy_tuned:.4f}")
print("\nClassification Report:")
print(report_tuned)

# If you computed base accuracy earlier as `accuracy`, compare:
try:
    print("\n--- Comparison with Base OS Model ---")
    print(f"Base OS Accuracy:       {accuracy:.4f}")
    print(f"Fine-Tuned OS Accuracy: {accuracy_tuned:.4f}")
    improvement = accuracy_tuned - accuracy
    print(f"Improvement:            {improvement:+.4f}")
except NameError:
    print("\n(Base model accuracy `accuracy` not found; skipping comparison.)")
